# 问题的提出

## 欧拉猜想

任何一个正整数的 $n$ 次方，至少需要 $n$ 个正整数的 $n$ 次方才能表示出来。

寻找 $n = 5$ 时的反例：

$x^5 + y^5 + z^5 + u^5 = w^5$

限定搜索范围在 $w < 200$ 内。

# 计算方法

## 1. 简单枚举法

这类问题的一般解法是枚举法，也就是通常说的“暴力破解”。比如，我们可以用以下方法写一个最简单的函数。

In [53]:
def power_sum_v1(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5 在 nmax 内的正整数解 '''
    
    for x in range(1, nmax):
        for y in range(1, nmax):
            for z in range(1, nmax):
                for u in range(1, nmax):
                    for w in range(1, nmax):
                        if x**5 + y**5 + z**5 + u**5 == w**5:
                            print(x, y, z, u, w)

如果要枚举所有在 $w < 200$ 内的解，只需要输入

```
power_sum_v1(200)
```

但是我们在测试程序的时候，通常会先用一些比较小的值，比如 `nmax = 20`，看看是否程序跑得够快。
所以会在运行程序的前面加上 `%timeit`，这样它会重复运行这个函数 7 遍，看平均运行一下需要多少时间。
如果它跑得很慢，我们就要修改和优化一下程序，而不是硬运行程序。

In [43]:
%timeit power_sum_v1(20)

2.25 s ± 24.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


遗憾的是，这个程序跑起来太慢了，
即便是`nmax=20` 而不是真正需要的 `nmax=200` ，我的机器上平均每次运行花了 2.3 秒的时间。
如果 `nmax=30` ，则要 20 秒的时间。
可想而知，`nmax = 200` 是根本跑不出来的。（推算了一下，对于 `nmax=200` 时我的机器要跑 3.5 天！）

| `nmax` | 运行时间 |
|--------|---------|
| 10     | 0.057 s |
| 20     | 2.3 s |
| 30     | 20 s |
| ...    | ... |
| 200    | 300000 s |



## 2. 利用对称性减少枚举数

优化的第一个思路是利用解的对称性来减少需要枚举的解的数量。我们先来罗列一些解的性质。

 1. 首先，因为我们考虑的是正整数解，$w$ 肯定比 $x$, $y$, $z$, $u$ 中的任何一个都要大。
    所以如果我们在枚举 $x = 100$ 时，根本不用考虑 $w = 1, 2, \dots 100$ 的情况。这样就可以省下好多时间。

 2. 第二，$x$, $y$, $z$, $u$ 是对称的，也就是说，我们可以随意把这四个数字进行位置交换，并不影响解的成立。
    所以我们可以把这四个变量排排序，$x \le y \le z \le u$。这样又可以省掉很多枚举的情况。
    
 3. 综合以上两点，我们判断，我们只需要枚举 $x \le y \le z \le u < w$ 的情况。把这5个数字排序之后，所需要枚举的情况数大概仅为原来的 $1/5! = 1/120$。
 
让我们来实现一下这个新的算法。
在内部的几层循环里，我们可以控制一下循环变量的初始值：
```
for x in range(1, nmax):
    for y in range(x, nmax): # 利用 y >= x
        for z in range(y, nmax): # 利用 z >= y
            for u in range(z, nmax): # 利用 u >= z
                for w in range(u + 1, nmax): # 利用 w > u
                    ...
```

In [54]:
def power_sum_v2(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5
        在 1 <= x <= y <= z <= u < w < nmax 内的正整数解 '''
    
    for x in range(1, nmax):
        for y in range(x, nmax): # 利用 y >= x
            for z in range(y, nmax): # 利用 z >= y
                for u in range(z, nmax): # 利用 u >= z
                    s = x**5 + y**5 + z**5 + u**5
                    for w in range(u + 1, nmax): # 利用 w > u
                        if s == w**5:
                            print(x, y, z, u, w)

### 最内层循环变量的优化

注意到，除了利用对称性，我们在最里面那层循环里也做了些手脚。

因为在最里面那层循环里只对 $w$ 进行变化，而 $x$, $y$, $z$, $u$ 都是不变的，所以我们可以把 $x^5+y^5+z^5+u^5$ 的值预先计算好，保留在一个变量 `s` 里，然后每次只要把 $w^5$ 和 $s$ 做比较即可。

```
s = x**5 + y**5 + z**5 + u**5
for w in range(u + 1, nmax): # 利用 w > u
    if s == w**5:
        ...
```

好，我们来试一下 `nmax = 20` 的情况。

In [55]:
%timeit power_sum_v2(20)

11.5 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


这次平均运行一次才 12ms， 比起之前的 2.3 s 快了 160 多倍！而且这个差距会继续会随着 `nmax` 的增加慢慢上升。

| `nmax` | 版本 1  | 版本 2 |
|--------|---------|---------|
| 10     | 0.057 s | 0.0006s | 
| 20     | 2.3 s   | 0.012s |
| 30     | 20 s    | 0.075s |
| 100    | ？      | 20s |

让我们试一个大一点的值`nmax = 20`

In [49]:
%timeit power_sum_v2(100)

20.3 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


对于 `nmax = 100`，它花了 20s。要是`nmax` 再上去，比如对于我们的 `nmax = 200`，这仍然不够快。所以我们要想想其它办法来改进这个算法。

## 3. 备忘录方法（查表法）

我们的第二个优化思路是这样的。注意到在这个函数中，我们计算了很多次自然数的5次方。这些计算都是重复和没有必要的。所以我们可以事先把1到200内自然数的5次方计算好，列张表。然后以后要用时，直接查这张表就好了。

在 Python 里这样一个表可以用列表（list）来实现。我们把这个表叫做 `pow5`，可以这么写来构造这张表：

```
pow5 = []
for x in range(nmax):
    pow5[x] = x**5
```

Python 还提供了一种更加简洁的语法来实现这个循环。这类语法叫做 list comprehension，其实就是一种简写循环的方式：

```
pow5 = [x**5 for x in range(nmax)]
```


列表以后还有个好处，就是我们可以省掉最内的一层循环。为什么呢？因为如果 $s = x^5+y^5+z^5+u^5$ 是一个数的5次方，那么它必然也在这个表内，所以我们可以用 Python 的语法

```
s in pow5
```

来判断 `s` 是否是一个数的5次方。

In [50]:
def power_sum_v3(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5
        在 1 <= x <= y <= z <= u < w < nmax 内的正整数解
        利用列表避免重复计算自然数的 5 次方的值 '''

    # 把 x = 0..nmax-1 的 x**5 的值做一个表格保存起来
    pow5 = [x**5 for x in range(nmax)]
        
    for x in range(1, nmax):
        for y in range(x, nmax):
            for z in range(y, nmax):
                for u in range(z, nmax):
                    s = pow5[x] + pow5[y] + pow5[z] + pow5[u]
                    if s in pow5:
                        print(x, y, z, u, s**(1/5))

对于 `nmax = 20` 这个版本只花了 1.9ms 就跑出结果了。

In [51]:
%timeit power_sum_v3(20)

1.9 ms ± 40.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


对于大一点的 `nmax = 100`，这个版本花了 3.6 s 就跑出结果了，比起上次的 22s 可好多了。

In [ ]:
%timeit power_sum_v3(100)

3.57 s ± 27.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


让我们想想有没有更好的办法。

## 4. 更好的查表法（字典）

其实我们判断 $s$ 是否是5次方数那步可以更有效。原因是如果 `pow5` 是一个列表 `list`，Python 在执行这句话时，

```
s in pow5
```

它仍然需要把这个列表里每个数字查一遍，才能判断 `s` 是否在列表 `pow5` 内；所以它的算法复杂度是 $O(n)$，其中 $n$ 代表列表里元素的个数。

但我们想想，如果有另外一个数组`t`，这个数组是以 `s` 作为其下标的，`t[s]` 的值是一般是 `False`，但是如果 $s$ 是5次方数，那么`t[s] = True`。这样我们就可以一下子判断 $s$ 是不是一个5次方数了对吧？

可惜的是，这种数组虽然存在，但是它的内存占用量太大了。$200^5$ 是个天文数字，没办法容纳弄那么大的数组。

### 字典

所幸的是， Python 还提供了一种更有效的数据结构，叫做 **字典**，可以处理这种稀疏的数组。

```
t = {}
for w in range(nmax):
    t[w**5] = True
```

对于一个五次方数

```
s in t
```

的值为 `True`，反之为`False`。

注意到，字典和列表用起来很像，只是字典初始化时用 `{}` 而不是 `[]`。

字典在内部实现时，它是一个哈希表（Hash table），比如我们这个字典里有200个值，它内部对应的真正数组可能就 1000 个值，`hash[0..999]`，当我们给它输入一个很大的数 `s` 时，它会根据 `s` 算一个 0 到 999 之间的下标 `i` （这个映射叫做哈希函数），然后把`t[s]` 的值对应在内部数组的 `hash[i]` 值上。

比如，一个最简单的办法是计算

```
i = s % 1000
```

当然，Python 内部的版本比这要复杂多了。这个例子只是说明哈希函数的计算一般是很快的，是个 $O(1)$ 的算法。不需要对字典内部的元素全部走一遍。当然如果不巧两个不同的 `s` 对应于同一个 `i`，就比较麻烦了。但我们不必操心哈希算法的具体实现。只要假定这是一个很快的算法就好了。

### 应用字典

在我们这个应用里，如果字典的输入值 `s` 是一个5次方数，我们还想知道它的5次方根是多少。我们可以利用字典的特性把 `w**5` 映射回为 `w`，而不是简单的 `True`，这样我们不仅可以判断是否`s`是一个5次方数，而且在`s`是一个5次方数时，可以迅速通过 `s` 反查 $\sqrt[5]{s}$ 的值

```
invpow5 = {}
for w in range(nmax):
    invpow5[w**5] = w
```

和列表一样，对于创建字典，我们也有一种简洁的写法：

```
invpow5 = {w**5:w for w in range(1, nmax)}
```

In [56]:
def power_sum_v4(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5
        在 1 <= x <= y <= z <= u < w < nmax 内的正整数解
        利用列表避免重复计算自然数的 5 次方的值
        利用字典判断一个数是否是 5 次方数 '''

    # 把 x**5 的值做一个表格
    pow5 = [x**5 for x in range(nmax)]
        
    # 把 w**5 -> w 也做一个反查的表格
    invpow5 = {w**5 : w for w in range(1, nmax)}

    for x in range(1, nmax):
        for y in range(x, nmax):
            for z in range(y, nmax):
                for u in range(z, nmax):
                    s = pow5[x] + pow5[y] + pow5[z] + pow5[u]
                    if s in invpow5: # 判断是否是 5 次方数
                        print(x, y, z, u, invpow5[s])

In [57]:
%timeit power_sum_v4(100)

421 ms ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


用了字典以后，程序又快了很多。在我的机器上，平均每次只用了约 0.4s 的时间。比上次的 3.5s 又好很多。

## 5. 循环优化

下一步的优化是对循环内的变量的优化。

我们看到，在最内层的循环里，这句话

```
for u in range(z, nmax):
    s = pow5[x] + pow5[y] + pow5[z] + pow5[u]
```

会重复计算前三项  `pow5[x] + pow5[y] + pow5[z]` 的值，然而这个值在对 `u` 的循环里是不变的，所以可以把它预先计算后提出循环。

In [29]:
def power_sum_v5(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5
        在 1 <= x <= y <= z <= u < w < nmax 内的正整数解
        最终优化版 '''

    # 把 x**5 的值做一个表格
    pow5 = [x**5 for x in range(nmax)]
        
    # 把 w**5 -> w 也做一个表格
    invpow5 = {w**5 : w for w in range(1, nmax)}

    for x in range(1, nmax):
        for y in range(x, nmax):
            for z in range(y, nmax):
                s_xyz = pow5[x] + pow5[y] + pow5[z]
                for u in range(z, nmax):
                    s = s_xyz + pow5[u]
                    if s in invpow5:
                        print(x, y, z, u, invpow5[s])

In [30]:
%timeit power_sum_v5(100)

226 ms ± 19.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


这次只要 0.23 秒了，又比前一次快了很多。

| `nmax` | 版本 1  | 版本 2   |  版本 3 |  版本 4 |  版本 5  |
|--------|---------|---------|---------|---------|---------|
| 100    | ？      | 20s     |   3.5s  |  0.42 s |  0.23s  |

## 6. 正式运行

经过数轮优化我们终于可以运行真正的 `nmax = 200` 的例子了。

In [31]:
power_sum_v5(200)

27 84 110 133 144


这次它真的跑出一个结果，而且它很快，在我的机器上，平均每次只花 3 秒钟，比最开始的 3.5 天快了近 100000 倍！很有成就感吧！

## 7. 对照文献结果

让我们对比一下论文的结果：

![Counterexample to Euler's conjecture on sums of like powers](https://i.niupic.com/images/2022/01/24/9Uc9.webp)

和我们的结果完全相同！

# 参考阅读

* [世界上最短的数学论文之一，关于费马大定理和欧拉猜想](https://mp.weixin.qq.com/s/Pd6R_GynMTRL1_n58ja5og)